<a href="https://colab.research.google.com/github/wajidminhas/AI-Agents/blob/main/reducer_chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain-google-genai langchain_core langgraph

create gemini api key to start chat model

In [2]:
from google.colab import userdata
google_api_key = userdata.get('GEMINI_API_KEY')

here chat model is being created with metadata, using gemini chat model flash 1.5 chat google generative ai

In [7]:
from os import name
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage
from pprint import pprint

messages = [AIMessage(content=f"how can i help you?", name="Model")]
messages.append(HumanMessage(content=f"i want to enjoy snow falling in northern pakistan", name="Wajid"))
messages.append(AIMessage(content=f"That sounds like a beautiful idea!", name="Model"))

for m in messages:
  # PrettyPrinter(f"{m.type}: {m.content} (from {m.name})")
  m.pretty_print()

llm: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=google_api_key)
result = llm.invoke(messages)
type(result)

================================== Ai Message ==================================
Name: Model

how can i help you?
================================ Human Message =================================
Name: Wajid

i want to enjoy snow falling in northern pakistan
================================== Ai Message ==================================
Name: Model

That sounds like a beautiful idea!


langchain_core.messages.ai.AIMessage